In [1]:
import sys
sys.path.append("../src")

In [2]:
from IPython.display import display, HTML
from Utils import *

In [3]:
rnu = RedditNetworkUtils()
rnu.read_comments_into_network("../data/RC_2013-02", "link_id", "parent_id", maxsize=5e3,
                               calculate_users_similarity=True)

Users (nodes): 979
Subreddits: 1010
Edges: 831


In [4]:
for n, data in rnu.ntw.nodes(data=True):
    for k,v in data.items():
        print(k,v)
    break

('polarity', 0)
('closeness_centrality', 0.0010224948875255625)
('w_outdeg', 0)
('w_indeg', 1)
('betweenness', 0.0)
('w_deg', 1)
('indeg', 1)
('subjectivity', 0)
('w_betweenness', 0.0)
('outdeg', 0)
('deg', 1)
